In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")
!pip install facenet-pytorch
# Clean up the notebook
clean_notebook()


Notebook cleaned.


# Face detection from Webcam

In [2]:
import cv2
import numpy as np
import torch
from facenet_pytorch import MTCNN

# Initialize the MTCNN model for face detection
mtcnn = MTCNN(keep_all=True, device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

# Our sketch generating function for the detected face
def sketch(image):
    # Convert image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Clean up image using Gaussian Blur
    img_gray_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    
    # Extract edges
    canny_edges = cv2.Canny(img_gray_blur, 10, 70)
    
    # Invert binarize the image
    _, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    return mask


if not cap.isOpened():
    print("Error: Could not access the webcam.")
    cap.release()
    raise RuntimeError("Webcam initialization failed.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break
    
    # Convert frame to RGB (MTCNN expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces using MTCNN
    boxes, _ = mtcnn.detect(rgb_frame)
    
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            # Ensure the bounding box is within the frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            # Extract the face region
            face_region = frame[y1:y2, x1:x2]
            
            # Apply sketch effect to the face region
            face_sketch = sketch(face_region)
            
            # Convert sketch to 3 channels to overlay on the original frame
            face_sketch_color = cv2.cvtColor(face_sketch, cv2.COLOR_GRAY2BGR)
            
            # Replace the face region in the original frame with the sketch
            frame[y1:y2, x1:x2] = cv2.resize(face_sketch_color, (x2 - x1, y2 - y1))
    
    # Display the frame with face sketches
    cv2.imshow('Live Sketch with Face Detection', frame)
    
    # Break loop on pressing 'Enter'
    if cv2.waitKey(1) == 13:  # 13 is the Enter Key
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()

Failed to grab frame.


[ WARN:0@2.984] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@2.984] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
